In [ ]:
import numpy as np
from glob import glob
import sys, os
import shutil

sys.path.append('..')
import util

_ROOT_DIR = '/mnt/data/new_disk/chenap/dataset/crop'
_OUTPUT_DIR = '/mnt/data/new_disk/chenap/dataset/seg_face_real'

if os.path.exists(_OUTPUT_DIR):
    shutil.rmtree(_OUTPUT_DIR)

os.makedirs(_OUTPUT_DIR)

img_fps = sorted(glob(os.path.join(_ROOT_DIR, '*.jpg')))

all_instances = set([os.path.basename(x).split('_')[0] for x in img_fps])

for iid, ins_id in enumerate(all_instances):
    
    output_dir = os.path.join(_OUTPUT_DIR, ins_id)
    os.makedirs(output_dir, exist_ok=True)
    
    cam2worlds = {}
    cam_int = {}

    all_imgs = glob(os.path.join(_ROOT_DIR, '%s_*.jpg'%(ins_id)))
    
    print('[%s] find %d views'%(ins_id, len(all_imgs)))
    
    for vid, img_fp in enumerate(all_imgs):
        
        img_fn = os.path.basename(img_fp)
        
        cam_params = np.load(img_fp.replace('.jpg', '.npy'), allow_pickle=True).item() 
        
        cam_world = np.linalg.inv(cam_params['extrinsics'])
        cam_world[:3, 2] *= -1
        cam_world[:3, 1] *= -1
                
        cam2worlds['seg_%02d.png'%(vid)] = cam_world
        cam_int['seg_%02d.png'%(vid)] = cam_params['intrinsic']

        shutil.copy(
            os.path.join(_ROOT_DIR, 'segmap', img_fn.replace('.jpg', '.png')), 
            os.path.join(output_dir, 'seg_%02d.png'%(vid)))
        shutil.copy(
            os.path.join(_ROOT_DIR, 'segmap', img_fn.replace('.jpg', '_vis.png')),
            os.path.join(output_dir, 'vis_%02d.png'%(vid)))
        
        shutil.copy(
            img_fp, os.path.join(output_dir, 'render_%02d.png'%(vid)))
        shutil.copy(
            img_fp.replace('.jpg', '.npy'), os.path.join(output_dir, 'cam_%02d.npy'%(vid)))
    
    np.save(os.path.join(output_dir, 'cam2world.npy'), cam2worlds)
    np.save(os.path.join(output_dir, 'intrinsics.npy'), cam_int)
    
    print('[%03d/%03d] %s'%(iid+1, len(all_instances), output_dir))
    

In [ ]:
import numpy as np
from glob import glob
import sys, os
import shutil

import cv2

sys.path.append('..')
import util

%matplotlib widget
import matplotlib.pyplot as plt

_ROOT_DIR = '/home/anpei/liury/data/facial-data/seg_face_real'

_TRGT_HW = 128

all_instances = sorted(glob(os.path.join(_ROOT_DIR, '[0-9]*')))
print('Total number of instances = ', len(all_instances))

cam_int_avg = []
cam_pose_r = []
cam_center = []

for iid, ins_dir in enumerate(all_instances[10:]):
    
    cam2worlds = np.load(os.path.join(ins_dir, 'cam2world.npy'),  allow_pickle=True).item()
    cam2worlds = np.asarray([cam2worlds[key] for key in cam2worlds.keys()])
        
    cam_ints = np.load(os.path.join(ins_dir, 'intrinsics.npy'),  allow_pickle=True).item()
        
    img_fp = list(cam_ints.keys())[0]
    seg_img = cv2.imread(os.path.join(ins_dir, img_fp), cv2.IMREAD_GRAYSCALE)
    height, width = seg_img.shape
    
    print(cam2worlds[0])
    
#     util.plot_camera_scene(cam2worlds)
#     plt.show()
            
    cur_cam_center =np.asarray([c2w[:3, 3].T for c2w in cam2worlds])
    cur_pose_r = [np.linalg.norm(c2w[:3, 3]) for c2w in cam2worlds]
    cur_cam_int = list(cam_ints.values())
        
    cam_pose_r.extend(cur_pose_r)
    cam_int_avg.extend(cur_cam_int)
    cam_center.extend(cur_cam_center)
    
    cur_pose_r = np.asarray(cur_pose_r)
    cur_cam_int = np.asarray(cur_cam_int)
    
    print(np.mean(cur_pose_r, axis=0))
    print(np.mean(cur_cam_center, axis=0))
    print(np.mean(cur_cam_int, axis=0))
    print('[%04d/%04d] =========================================='%(iid, len(all_instances)))
    
#     break
        
cam_pose_r = np.asarray(cam_pose_r)
cam_int_avg = np.asarray(cam_int_avg)

print(cam_pose_r.shape, cam_int_avg.shape)

cam_pose_r = np.mean(cam_pose_r, axis=0)
cam_center = np.mean(cam_center, axis=0)
cam_int_avg = np.mean(cam_int_avg, axis=0)

print(cam_pose_r)
print(cam_center)
print(cam_int_avg)

In [ ]:
cam_int = np.eye(4)
cam_int[:3, :3] = cam_int_avg

np.save(os.path.join(_ROOT_DIR, 'intrinsics_avg.npy'), cam_int)

In [ ]:
%matplotlib widget
import util
import matplotlib.pyplot as plt

cam2worlds = np.asarray([cam2worlds[key] for key in cam2worlds.keys()])
util.plot_camera_scene(cam2worlds)
plt.show()

In [ ]:
print(np.asarray(cam_int).shape)
print(cam_int[0])

In [ ]:
import numpy as np
from glob import glob
import os
import shutil

_ROOT_DIR = '/data/new_disk2/liury/data/facial-data/seg_face_0702'

img_fps = glob(os.path.join(_ROOT_DIR, '*', 'seg_vis_*.png'))

cam_fps = glob(os.path.join(_ROOT_DIR, '*', 'cameras.npy'))

zRange = []

for idx, cam_fp in enumerate(cam_fps):
#     print('| ', cam_fp)
    instance_dir = os.path.dirname(cam_fp)
    
    cam_params = np.load(cam_fp, allow_pickle=True).item()     
    world2cam = cam_params['extrinsics']
        
    zRange = np.asarray(cam_params['zRange'])
    cam2world = [np.linalg.inv(x) for x in world2cam]
    cam_int = [cam_params['intrinsic']]*len(cam2world)
    
    img_fps = ['seg_vis_%02d.png'%(x) for x in range(len(cam2world))]
    cam2world = dict(zip(img_fps, cam2world))
    cam_int = dict(zip(img_fps, cam_int))
      
#     print(type(cam_int), type(cam2world))
#     print(cam2world['seg_vis_00.png'])
#     print(cam_int['seg_vis_00.png'])
    
#     print(cam2world)
    
#     print('R = ', np.linalg.norm(cam2world[:3, 3]))
    
    output_fp = os.path.join(instance_dir, 'cam2world.npy')
    np.save(output_fp, cam2world)
    
    output_fp = os.path.join(instance_dir, 'zRange.npy')
    np.save(output_fp, zRange)
    
    output_fp = os.path.join(instance_dir, 'intrinsics.npy')
    np.save(output_fp, cam_int)
    
    print('[%03d/%03d] %s'%(idx, len(cam_fps), output_fp))

In [ ]:
zRange = np.asarray(zRange)
print('*** zRange = ', zRange.shape)

zRange[:, :, 0] = np.min(zRange)
zRange[:, :, 1] = np.max(zRange)

for idx, cam_fp in enumerate(cam_fps):
    output_fp = os.path.join(instance_dir, 'zRange.npy')
    np.save(output_fp, zRange[idx])

In [ ]:
import numpy as np
from glob import glob
import os
import shutil

_ROOT_DIR = '/data/anpei/facial-data/seg_face_0417'

# img_fps = glob(os.path.join(_ROOT_DIR, '*', 'labal_*.png'))

# for idx, img_fp in enumerate(img_fps):
#     dst_fp = img_fp.replace('labal', 'seg')
#     print('[%d/%d]'%(idx, len(img_fps)), img_fp, dst_fp)
#     shutil.move(img_fp, dst_fp)

zRange = []

cam_fps = glob(os.path.join(_ROOT_DIR, '*', 'cameras.npy'))

for idx, cam_fp in enumerate(cam_fps):
#     print('| ', cam_fp)
    instance_dir = os.path.dirname(cam_fp)
    cam_params = np.load(cam_fp, allow_pickle=True).item()    
    world2cam = cam_params['extrinsics']
    
#     ins_param = cam_params['instance']
    
#     print(cam_params.keys(), ins_param.shape)
    
#     print(world2cam[idx])
    
#     zRange = np.asarray(cam_params['zRange'])

    zRange.append(np.asarray(cam_params['zRange']))

    cam_int = np.asarray(cam_params['intrinsic'])
#     face_params = np.squeeze(cam_params['instance'])
    
#     print(zRange)
    
#     print(cam_int)
#     for cam_ext in world2cam:
#         print(cam_ext)
#         print('----------------------------------')


#     cam2world = np.asarray([np.linalg.inv(x) for x in world2cam])    
#     output_fp = os.path.join(instance_dir, 'cam2world.npy')
#     np.save(output_fp, cam2world)
    
#     output_fp = os.path.join(instance_dir, 'zRange.npy')
#     np.save(output_fp, zRange)
    
#     output_fp = os.path.join(instance_dir, 'params.npy')
#     np.save(output_fp, face_params)
    
    print('[%03d/%03d] %s'%(idx, len(cam_fps), output_fp))
        

In [ ]:
zRange = np.asarray(zRange)
print('*** zRange = ', zRange.shape)

zRange[:, :, 0] = np.min(zRange)
zRange[:, :, 1] = np.max(zRange)

print(np.unique(zRange))

for idx, cam_fp in enumerate(cam_fps):
    output_fp = os.path.join(instance_dir, 'zRange.npy')
    np.save(output_fp, zRange[idx])

In [ ]:
np.save('cam2world.npy', np.asarray(cam2world))
np.save('extrinsics.npy', np.asarray(world2cam))


cc = np.load('cam2world.npy')
print(cc[0].shape)

In [ ]:
import os
from PIL import Image

img = Image.open(os.path.join('images', '00000_seg_02_04.png'))
print(img.size)

In [ ]:
import glob

imgs = sorted(glob.glob(os.path.join('images', '00000_seg_*.png')))
print(imgs, len(imgs))

In [ ]:
from glob import glob
import os
tot_imgs = glob(os.path.join('images', '*.png'))
tot_imgs = set([os.path.basename(x)[:5] for x in tot_imgs])
tot_imgs = [int(x) for x in tot_imgs]

print(len(tot_imgs))

for x in tot_imgs:
    print(x)

In [ ]:
import numpy as np
P = np.load('cam2world.npy')
print(P.shape)

In [ ]:
import numpy as np

P = np.load('/data/anpei/facial-data/seg_face_8000/cam2world.npy')
# print(P[0])

for i in range(P.shape[0]):
    R = P[0, :3, :3]
    T = P[0, :3, 3]

    print(np.linalg.det(R), np.linalg.norm(T))

In [ ]:
import numpy as np
from mpl_toolkits import mplot3d

import matplotlib.pyplot as plt

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.set_xlim([-1.5,1.5])
ax.set_ylim([-1.5,1.5])
ax.set_zlim([-1.5,1.5])

_r = 1.1165075011366656

_v = 1.0
_omega = 20

t = np.linspace(0, 2, num=100)
z = -_r + t * _v

r = np.sqrt(_R**2-z**2)
x = r * np.cos(_omega * t)
y = r * np.sin(_omega * t)

ax.plot3D(x, y, z, 'gray')

In [ ]:
print(x.shape, y.shape, z.shape)
_cam_pos = (np.stack([x, y, z])).T
print(_cam_poses.shape)

_cam_target = np.zeros((1,3))
_cam_up = np.asarray([0.0, 1.0, 0.0])

CAM_X = []
CAM_Y = []
CAM_Z = []

_cam2world = []

for idx in range(10):
    cam_pos = _cam_pos[idx, :]
    cam_dir = (_cam_target - cam_pos)
    cam_dir = cam_dir / np.linalg.norm(cam_dir)
    cam_right = np.cross(_cam_up, cam_dir)
    cam_right = cam_right / np.linalg.norm(cam_right)
    cam_up = np.cross(cam_dir, cam_right)
    cam_up = cam_up / np.linalg.norm(cam_up)
    
    cam_R = np.concatenate([cam_right, cam_up, cam_dir, _cam_target], axis=0)
    cam_P = np.ones((4, 1))
    cam_P[:3,] = np.expand_dims(cam_pos, 1)
        
    _cam2world.append(np.concatenate([cam_R, cam_P], axis=1))

_cam2world = np.asarray(_cam2world)
print(_cam2world.shape)



In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
# ax.set_xlim([-2.5,1.5])
# ax.set_ylim([-1.5,1.5])
# ax.set_zlim([-1.5,1.5])

ax.quiver(_cam_pos[:, 0],_cam_pos[:, 1],_cam_pos[:, 2], CAM_X[:, 0], CAM_X[:, 1], CAM_X[:, 2], 'red')

In [ ]:
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure()
ax = fig.gca(projection='3d')

# Make the grid
x, y, z = np.meshgrid(CAM_X,
                      CAM_Y,
                      CAM_Z)

# Make the direction data for the arrows
u = np.sin(np.pi * x) * np.cos(np.pi * y) * np.cos(np.pi * z)
v = -np.cos(np.pi * x) * np.sin(np.pi * y) * np.cos(np.pi * z)
w = (np.sqrt(2.0 / 3.0) * np.cos(np.pi * x) * np.cos(np.pi * y) *
     np.sin(np.pi * z))

print(x.shape, u.shape)

ax.quiver(x, y, z, u, v, w, length=0.1, normalize=True)

plt.show()

In [ ]:
import data_util
img = data_util.load_gray('/data/anpei/facial-data/seg_face_2000/09667/seg_19.png', sidelength=128)

In [ ]:
import numpy as np
print(np.max(img), np.min(img))

In [ ]:
import random

print(np.random.randint(0, 25, 1)[0])

In [ ]:
from glob import glob
import os
from pathlib import Path
import shutil
import numpy as np

import random

import cv2

_ROOT_DIR = '/data/anpei/facial-data/seg_video/video'
img_fps = sorted(glob(os.path.join(_ROOT_DIR, '*.png')))

output_dir = os.path.join(_ROOT_DIR, '..', 'fake')

bin_size = (len(img_fps) // 25)

cams = np.load('/data/anpei/facial-data/seg_video/0000/cams.npy', allow_pickle=True)

for idx in range(25):
    img_idx = idx*bin_size+np.random.randint(0, bin_size, 1)[0]
    
    img_fp = img_fps[img_idx]
    
    cam2world = np.expand_dims(cams[img_idx]['extrinsic'], axis=0)
#     cam2world[:3, 3] /= 100.0
#     print(np.linalg.det(cam2world[:3, :3]))
    
    trgt_dir = os.path.join(output_dir, '%04d'%(idx*4+3))
    
#     os.makedirs(trgt_dir, exist_ok=True)
    
    trgt_fp = os.path.join(trgt_dir, 'seg_00.png')
    
    img = cv2.imread(img_fp, cv2.IMREAD_UNCHANGED)
    H, W, C = img.shape    
    seg = img[:, W//2:, 0]
    
    cv2.imwrite(trgt_fp, seg)
    np.save(os.path.join(trgt_dir, 'cam2world.npy'), cam2world)
    
    print(im_id, trgt_fp, cam2world.shape, seg.shape)
    

In [ ]:
import numpy as np
from glob import glob
import os

_ROOT_DIR = '/data/anpei/facial-data/seg_video/0000'

cam_fp = os.path.join(_ROOT_DIR, 'cams.npy')

cam_exts = np.load(cam_fp, allow_pickle=True)

for idx, cam in enumerate(list(cam_exts)):
    cam_ext = cam['extrinsic']
    R_ext = cam_ext[:3, :3]
#     print(np.linalg.det(cam_ext))
    cam2world = np.linalg.inv(cam['extrinsic'])
    f = 1.0
    fx, fy = cam['scale']
    
    
#     int_fp = os.path.join('%04d'%)
    print(idx, fx, fy)
#     print(cam2world.shape, cam['extrinsic'].shape)
#     print(cam['extrinsic'])
    print(cam2world)
#     R = cam2world[:3, :3]
#     T = cam2world[:3, 3].T
    print(np.linalg.det(R))

In [ ]:
from scipy.spatial.transform import Rotation as R

_ROOT_DIR = '/data/anpei/facial-data/seg_face_new/'
_OUTPUT_DIR = '/data/anpei/facial-data/seg_video/fake/'
ind_idx = glob(os.path.join(_ROOT_DIR, '[0-9]*'))

slc_idx = random.choices(ind_idx, k=25)
print(len(slc_idx))

# transform = torchvision.transforms.RandomAffine(10, )

for idx, src in enumerate(slc_idx):
    out_dir = os.path.join(_OUTPUT_DIR, '%04d'%(idx*4+2))
    os.makedirs(out_dir, exist_ok=True)
    cam2worlds = np.load(os.path.join(src, 'cam2world.npy'))
    
    cam_idx = random.choice(list(range(25)))
    cam2world = cam2worlds[idx]
    cam_R = cam2world[:3, :3]
    cam_T = cam2world[:3, 3]
    
    rot = (R.from_euler('xyz', (np.random.rand(3)-0.5)*10, degrees=True)).as_dcm()
    cam2world[:3, :3] = rot.dot(cam_R)
    cam2world[:3, 3] = cam_T + (np.random.rand(3)-0.5)*0.2
    
    cam2world = np.expand_dims(cam2world, axis=0)
#     print(cam2world)
    
    img_fp = os.path.join(src, 'seg_%02d.png'%(cam_idx))
    
    shutil.copyfile(img_fp, os.path.join(out_dir, 'seg_00.png'))
    np.save(os.path.join(out_dir, 'cam2world.npy'), cam2world)
    
    print(idx, img_fp)

In [ ]:
from scipy.spatial.transform import Rotation as R

rot = (R.from_euler('xyz', (np.random.rand(3)-0.5)*10, degrees=True)).as_dcm()

# print(dir(rot))
# rot = rot.as_matrix()

print(rand_rot, rot)

In [ ]:
import random
_ROOT_DIR = '/data/anpei/facial-data/seg_face_new'
_OUTPUT_DIR = '/data/anpei/facial-data/seg_video/0000/'
ind_idx = glob(os.path.join(_ROOT_DIR, '[0-9]*'))
print(len(ind_idx))

slc_ins = random.choices(ind_idx, k=25)
print(slc_cams)

for ins in slc_ins:
    output_dir = os.path.join


In [ ]:
cam_path = '/data/anpei/facial-data/seg_face_2000_depth/09616/cam2world.npy'
cam_params = np.load(cam_path)

for cam_param in cam_params:
    R = cam_param[:3, :3]
    print(cam_param)
    print(np.linalg.det(R))

In [ ]:
_ROOT_DIR = '/data/anpei/facial-data/seg_video/0000/images'
img_dirs = glob(os.path.join(_ROOT_DIR, '[0-9]*'))

for img_dir in img_dirs:
    print(img_dir)
    shutil.copyfile('/data/anpei/facial-data/seg_video/0000/cam2world.npy', os.path.join(img_dir, 'cam2world.npy'))

In [ ]:
print(cams)

In [ ]:
cam_fp = os.path.join(_ROOT_DIR, 'cams.npy')
cams = np.load(cam_fp, allow_pickle=True)

cam2world = np.asarray(
    [np.linalg.inv(cam['extrinsic']) for cam in cams], dtype=np.float32)

cam2world[:, :3, 3] /= 100.0

print(cam2world)

In [ ]:
import json
import wget
import os


slc_ctg = ['"Coats & Outerwear"', '"Dresses"', '"Sleepwear"', '"Shirts & Tops"', '"Sweaters"', ] 

output_dir = '/data/anpei/facial-data/mvc_fashion'

os.makedirs(output_dir, exist_ok=True)

all_urls = []

with open('mvc_info.json') as json_file:
    data = json.load(json_file)
    
    for item in data:
         if (item['productGender'] == '"Womens"') and (item['category'] in slc_ctg):
            all_urls.append(item['image_url_4x'])

with open('all_urls.txt', 'w') as f:
    f.writelines('\n'.join(all_urls))

In [ ]:
print(data[0].keys())

In [ ]:
import os
from glob import glob
from PIL import Image, ImageOps
import sys

import matplotlib.pyplot as plt

import torchvision.transforms.functional as F

data_dir = '/data/anpei/facial-data/bag_imgs/images'
img_fps = glob(os.path.join(data_dir, '*'))

output_dir = '/data/anpei/facial-data/bag_imgs/resized'
os.makedirs(output_dir, exist_ok=True)

print(len(img_fps))



In [ ]:
failed_list = []
desired_size = 1024

for idx, im_pth in enumerate(img_fps):

    try:
        im = Image.open(im_pth).convert("RGB")
        old_size = im.size  # old_size[0] is in (width, height) format

        ratio = float(desired_size)/max(old_size)
        new_size = tuple([int(x*ratio) for x in old_size])
        # use thumbnail() or resize() method to resize the input image

        # thumbnail is a in-place operation

        # im.thumbnail(new_size, Image.ANTIALIAS)

        im = im.resize(new_size, Image.ANTIALIAS)
        # create a new image and paste the resized on it
        
        
        padding = ((desired_size-im.size[0])//2, (desired_size-im.size[1])//2)
        new_im = F.pad(im, padding, padding_mode='edge')
        new_im = F.center_crop(new_im, desired_size)
        
        output_fp = os.path.join(output_dir, '%04d.jpg'%(idx))
        new_im.save(output_fp)
        print('[%d/%d] %s'%(idx, len(img_fps), output_fp), new_im.size)

#         plt.imshow(new_im)
#         plt.show()
        
    except:
        print("Unexpected error:", sys.exc_info())
        failed_list.append(im_pth)

print('Failed: ', failed_list)